In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.nhl_scraper import time_remaining, nhl_scraper, shot_scraper2, get_processed_data, get_skater_stats, add_skaters_on_ice

In [3]:
import pandas as pd
from nhlpy import NHLClient
from datetime import datetime

In [4]:
game_id = 2024021036

In [5]:
shots_df,blocks_df,misses_df,goals_df,hits_df,give_take_df,faceoffs_df,pens_df,shifts_df,players_df,games_df,teams_df = nhl_scraper([game_id])

In [7]:
from src.config import MODELS_DIR
import joblib

model = joblib.load(MODELS_DIR / "xgb_v1.pkl")

c:\Users\nolan\anaconda3\envs\reports\Lib\pickle.py:1760: UserWarning: [21:06:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [8]:
temp_df, time_df = shot_scraper2([game_id])
temp_df = get_skater_stats(temp_df)
processed_df = get_processed_data(temp_df)

processed_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation,target
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,D,R,0.0464,L,0.898198,40.91,RL,EV,0
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,R,L,0.1260,L,0.898198,49.76,LL,EV,0
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,L,R,0.1319,L,0.898198,26.57,RL,EV,0
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,C,L,0.1420,L,0.905881,43.53,LL,EV,0
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,D,L,0.0446,L,0.898198,10.92,LL,EV,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,L,R,0.1456,L,0.898198,-6.34,RL,EV,0
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,D,R,0.0464,L,0.898198,6.01,RL,PP,1
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,C,L,0.1111,L,0.905881,-15.15,LL,SH,0
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,D,R,0.0372,L,0.905881,-1.01,RL,EV,0


In [18]:
full_shots_df = add_skaters_on_ice(processed_df, time_df, shifts_df)
full_shots_df.columns

Index(['home', 'last_play', 'rebound', 'rush', 'home_skaters', 'away_skaters',
       'x_coord', 'y_coord', 'shot_type', 'zone', 'position', 'shooter_hand',
       'shooting_pct', 'glove_hand', 'save_pct', 'angle', 'shot_on_glove',
       'situation', 'target', 'period', 'time', 'time_seconds', 'game_seconds',
       'home_players', 'away_players'],
      dtype='object')

In [20]:
preds = model.predict_proba(full_shots_df.drop(['period', 'time', 'time_seconds', 'game_seconds',
       'home_players', 'away_players'], axis=1))
xG = preds[:,1]

In [21]:
full_shots_df["xG"] = xG

In [22]:
full_shots_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,...,shot_on_glove,situation,target,period,time,time_seconds,game_seconds,home_players,away_players,xG
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,...,RL,EV,0,1,00:43,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ...",0.008904
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,...,LL,EV,0,1,01:39,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.204576
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,...,RL,EV,0,1,01:56,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.146474
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,...,LL,EV,0,1,02:06,126,126,"[8474141, 8475718, 8475765, 8475852, 8482762, ...","[8477365, 8478413, 8480045, 8480802, 8481524, ...",0.052182
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,...,LL,EV,0,1,02:57,177,177,"[8474141, 8475279, 8475852, 8479337, 8481542, ...","[8477365, 8478413, 8480045, 8482097, 8482671, ...",0.029260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,...,RL,EV,0,3,13:26,806,3206,"[8475852, 8477946, 8481607, 8482078, 8482762]","[8480045, 8480802, 8480839, 8481524, 8483468]",0.130279
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,...,RL,PP,1,3,14:31,871,3271,"[8474141, 8475225, 8475718, 8475852, 8476979, ...","[8475722, 8477365, 8478043, 8479359, 8480045, ...",0.077886
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,...,LL,SH,0,3,18:32,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ...",0.028868
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,...,RL,EV,0,3,19:15,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ...",0.015626
